# Tool Library

f

In [1]:
from tool_library.library import ToolLibrary
from tool_library.tools import *
from tool_library.factory import *


/home/adam/miniconda3/envs/kalavai/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-01-16 14:01:51,895	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


ImportError: cannot import name 'FastApiToolBelt' from 'tool_library.factory' (/home/adam/Documents/repos/tool-library/tool_library/factory.py)

In [ ]:
tool_library = ToolLibrary()

In [ ]:
from pydantic import BaseModel

class ExampleModel(BaseModel):
    field1: int
    field2: str


# Example of a simple function
def add_numbers(a: int, b: int) -> int:
    return a + b

# Example of a function that takes a Pydantic model
def process_model(data: ExampleModel):
    # Process the data
    return {"processed": True}

# Create an instance of the ToolLibrary
tool_library = ToolLibrary()

# Register the tools
tool_library.register_tool("Add Numbers", "Adds two numbers", add_numbers)
tool_library.register_tool("ProcessModel", "Processes data using a Pydantic model", process_model)

In [ ]:
tool_library.find_tools("Process")
tool_library.execute_tool("ProcessModel", {"field1": "A", "field2": 3})

{'processed': True}

In [ ]:
@ray.remote
def my_ray_function(x:str):
    return x * x

RayTool(
    "a","b",
    my_ray_function
)

tool_library.register_tool(
    name = "ray_test2",
    description= "ray_test",
    function = my_ray_function
)

In [ ]:
tool_library.find_tools("ray")

[{'name': 'ray_test2', 'params': None}]

In [ ]:
tool_library.register_api_tool("http://127.0.0.1:8000")
tool_library.find_tools(
    "list"
)

library.execute_tool("POST /v1/list_deepsparse_deployments", params={"namespace":"petals-backend2"})

NameError: name 'FastApiToolBelt' is not defined

In [ ]:
library.tools.keys()

dict_keys(['GET /v1/get_cluster_capacity', 'GET /v1/get_cluster_labels', 'POST /v1/get_node_labels', 'POST /v1/get_node_stats', 'POST /v1/get_nodes_cost', 'POST /v1/get_namespaces_cost', 'POST /v1/deploy_deepsparse_model', 'POST /v1/delete_deepsparse_model', 'POST /v1/list_deepsparse_deployments', 'POST /v1/deploy_generic_model'])

In [ ]:
import requests
from typing import Dict, Any, List
from prance import ResolvingParser

class FastApiRouteTool:
    def __init__(self, name, description, method, endpoint_url, params):
        self.name = name
        self.description = description
        self.method = method.upper()
        self.endpoint_url = endpoint_url
        self.params = params

    def execute(self, payload: Dict, auth=None) -> Any:
        headers = {}
        if auth:
            headers['Authorization'] = auth

        try:
            if self.method == 'GET':
                response = requests.get(self.endpoint_url, params=payload, headers=headers)
            elif self.method == 'POST':
                response = requests.post(self.endpoint_url, json=payload, headers=headers)
            # Add other HTTP methods as necessary
            response.raise_for_status()
            return response.json()
        except requests.RequestException as e:
            return {"error": str(e), "response": e.response.text}



class FastApiToolFactory:
    def __init__(self, service_url: str, add_routes:List[str]=None):
        self.service_url = service_url
        self.tools = []
        self.schema = None
        self.add_routes = add_routes
        self.introspect_service()

    def fetch_openapi_schema(self) -> None:
        try:
            # Use prance to load and resolve the OpenAPI specification
            parser = ResolvingParser(f"{self.service_url}/openapi.json")
            self.schema = parser.specification
        except Exception as e:
            raise ConnectionError(f"Error resolving OpenAPI schema: {e}")

    def introspect_service(self) -> None:
        if not self.schema:
            self.fetch_openapi_schema()

        for path, methods in self.schema.get('paths', {}).items():
            if self.add_routes and path not in self.add_routes:
                continue

            for method, details in methods.items():
                params = details
                self._create_tool_from_endpoint(path, method, details, params)

    def _create_tool_from_endpoint(self, path: str, method: str, details: Dict[str, Any], params: List[Dict[str, Any]]) -> None:
        tool_name = f"{method.upper()} {path}"
        description = details.get('summary', 'No description available')
        endpoint_url = f"{self.service_url}{path}"
        self.tools.append(FastApiRouteTool(tool_name, description, method, endpoint_url, params))

In [ ]:
belt = FastApiToolFactory(
    service_url="http://127.0.0.1:8000"
)
belt.introspect_service()

for t in  belt.tools:
    print(t.name, t.method, t.endpoint_url, t.params)

get
get
post
post
post
post
post
post
post
post
GET /v1/get_cluster_capacity GET http://127.0.0.1:8000/v1/get_cluster_capacity {'summary': 'Cluster Capacity', 'operationId': 'cluster_capacity_v1_get_cluster_capacity_get', 'responses': {'200': {'description': 'Successful Response', 'content': {'application/json': {'schema': {}}}}}}
GET /v1/get_cluster_labels GET http://127.0.0.1:8000/v1/get_cluster_labels {'summary': 'Cluster Labels', 'operationId': 'cluster_labels_v1_get_cluster_labels_get', 'responses': {'200': {'description': 'Successful Response', 'content': {'application/json': {'schema': {}}}}}}
POST /v1/get_node_labels POST http://127.0.0.1:8000/v1/get_node_labels {'summary': 'Node Labels', 'operationId': 'node_labels_v1_get_node_labels_post', 'requestBody': {'content': {'application/json': {'schema': {'properties': {'node_names': {'items': {'type': 'string'}, 'type': 'array', 'title': 'Node Names'}}, 'type': 'object', 'title': 'NodeLabelsRequest'}}}, 'required': True}, 'response

In [ ]:
from prance import ResolvingParser

# Load and resolve an OpenAPI specification file
parser = ResolvingParser(belt.service_url+"/openapi.json")

# Now, parser.specification holds the fully resolved spec as a dictionary
resolved_spec = parser.specification

# You can access paths and components directly
paths = resolved_spec.get

In [ ]:
resolved_spec["paths"]

{'/v1/get_cluster_capacity': {'get': {'summary': 'Cluster Capacity',
   'operationId': 'cluster_capacity_v1_get_cluster_capacity_get',
   'responses': {'200': {'description': 'Successful Response',
     'content': {'application/json': {'schema': {}}}}}}},
 '/v1/get_cluster_labels': {'get': {'summary': 'Cluster Labels',
   'operationId': 'cluster_labels_v1_get_cluster_labels_get',
   'responses': {'200': {'description': 'Successful Response',
     'content': {'application/json': {'schema': {}}}}}}},
 '/v1/get_node_labels': {'post': {'summary': 'Node Labels',
   'operationId': 'node_labels_v1_get_node_labels_post',
   'requestBody': {'content': {'application/json': {'schema': {'properties': {'node_names': {'items': {'type': 'string'},
         'type': 'array',
         'title': 'Node Names'}},
       'type': 'object',
       'title': 'NodeLabelsRequest'}}},
    'required': True},
   'responses': {'200': {'description': 'Successful Response',
     'content': {'application/json': {'schema'

In [ ]:
belt.schema["paths"]["/v1/deploy_generic_model"]

{'post': {'summary': 'Deploy Ray Model',
  'operationId': 'deploy_ray_model_v1_deploy_generic_model_post',
  'requestBody': {'content': {'application/json': {'schema': {'$ref': '#/components/schemas/GenericDeploymentRequest'}}},
   'required': True},
  'responses': {'200': {'description': 'Successful Response',
    'content': {'application/json': {'schema': {}}}},
   '422': {'description': 'Validation Error',
    'content': {'application/json': {'schema': {'$ref': '#/components/schemas/HTTPValidationError'}}}}}}}

In [ ]:
api_tool= FastApiRouteTool(
    "test","test","POST","http://127.0.0.1:8000/v1/deploy_generic_model", params={}
)

In [ ]:
belt = FastApiToolFactory(
    service_url="http://127.0.0.1:8000"
)
belt.introspect_service()

In [ ]:
for t in  belt.tools:
    print(t.name, t.method, t.endpoint_url, t.params)

GET /v1/get_cluster_capacity GET http://127.0.0.1:8000/v1/get_cluster_capacity []
GET /v1/get_cluster_labels GET http://127.0.0.1:8000/v1/get_cluster_labels []
POST /v1/get_node_labels POST http://127.0.0.1:8000/v1/get_node_labels []
POST /v1/get_node_stats POST http://127.0.0.1:8000/v1/get_node_stats []
POST /v1/get_nodes_cost POST http://127.0.0.1:8000/v1/get_nodes_cost []
POST /v1/get_namespaces_cost POST http://127.0.0.1:8000/v1/get_namespaces_cost []
POST /v1/deploy_deepsparse_model POST http://127.0.0.1:8000/v1/deploy_deepsparse_model []
POST /v1/delete_deepsparse_model POST http://127.0.0.1:8000/v1/delete_deepsparse_model []
POST /v1/list_deepsparse_deployments POST http://127.0.0.1:8000/v1/list_deepsparse_deployments []
POST /v1/deploy_generic_model POST http://127.0.0.1:8000/v1/deploy_generic_model []


In [ ]:
lister = belt.tools[-2]

In [ ]:
lister.execute(payload={"namespace":"petals-backend"})

{'petals-deployment': {'replicas': 1,
  'available_replicas': None,
  'unavailable_replicas': 1,
  'ready_replicas': None,
  'paused': None},
 'petals-service': {'cluster_ip': '10.43.29.157', 'ports': [[30294, 31330]]}}